In [2]:
import jieba;
jieba.enable_paddle();
#ya,I think jieba are kinda magical

import copy;
import math;
import json;
import requests;
from random import randrange
from bs4 import BeautifulSoup;



things={};
with open("./output415-Copy1.txt",encoding='utf-8') as ifile:
    things=json.loads(ifile.read());
print("done!");

Paddle enabled successfully......


done!


In [10]:
print(things["data"][1]);

{'name': '蝙蝠俠', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/%E8%9D%99%E8%9D%A0%E4%BF%A0-the-batman-10904', 'when': '2022-03-04', 'desc': '《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版《蝙蝠俠》中飾演這位DC超級英雄，電影是由《猩球崛起：終極決戰》導演麥特李維斯（Matt Reeves）所執導。', 'class': ['動作', '犯罪']}


In [42]:

def scan_doc(inname,inlist):
    doc={
        "name":inname,
        "total_word_count":0,
        "stat":{}# word -> {count,idf}
    };
    
    #scan thorogh list
    for word in inlist:
        if word in doc["stat"]:
            doc["stat"][word]["count"]+=1;
        else:
            doc["stat"][word]={};
            doc["stat"][word]["name"]=word;
            doc["stat"][word]["source"]=inname;
            doc["stat"][word]["count"]=1;
        doc["total_word_count"]+=1;
    #end of loop
   
    return doc;
#end of scan_doc

def scan_doc_list(inlist):
    docs={
        "total_doc_count":0,
        "total_word_count":0,
        "stat":{},# word -> {count,idf}
        "doc":{}#line -> doc
    };
    
    #scan every doc in lista
    for line,line_of_list in inlist.items():
        doc=scan_doc(line,line_of_list);
        docs["doc"][line]=doc;
        #merge stat loop
        for word,info in doc["stat"].items():
            if word in docs["stat"]:
                docs["stat"][word]["count"]+=doc["stat"][word]["count"];
                docs["stat"][word]["doc_count"]+=1;
            else:
                docs["stat"][word]={};
                docs["stat"][word]["name"]=word;
                docs["stat"][word]["count"]=copy.deepcopy(doc["stat"][word]["count"]);
                docs["stat"][word]["doc_count"]=1;
        #end of merge loop
        docs["total_doc_count"]+=1;
        docs["total_word_count"]+=doc["total_word_count"];
    #end of scan
    
    for word,info in docs["stat"].items():
        info["idf"]=math.log((docs["total_doc_count"])/(info["doc_count"]),10);
    #end of idf cal
    
    for word,doc in docs["doc"].items():
        for word,info in doc["stat"].items():
            tr=(info["count"])/(doc["total_word_count"]);
            idf=docs["stat"][word]["idf"];
            info["tfidf"]=(tr)*(idf);
        #end of a word
    #end of a doc
    return docs;
#end of scan_doc_list


class prediction:
    def __init__(self,indataset):
        self.vector={};#each contain word,index and their idf value
        self.map=[];#each contain list of idf,and class
        
        doc_list={};
        for i,entry in enumerate(indataset):
            line=entry["name"]+entry["desc"];
            cut_list=list(jieba.cut(entry["name"]))+list(jieba.cut(entry["desc"]));
            doc_list[line]=cut_list;
        #end of making thing from list
        
        #yes,code resused
        docs=scan_doc_list(doc_list);
        
        #converter them into vector
        for entry in docs["stat"].values():
            self.vector[entry["name"]]={"idf":entry["idf"]};
        
        for entry in docs["doc"].values():
            point={};
            for word,info in entry["stat"].items():
                point[word]=info["tfidf"];
            self.map.append(point);
                
        print(self.map);
        return;
    def predict():
        return;
    #end of __init
#end of prediction

prediction(things["data"][0:1])

[{'小小': 0.0, '兵': 0.0, '2': 0.0, '：': 0.0, '格魯': 0.0, '的': 0.0, '崛起': 0.0, '今年': 0.0, '暑假': 0.0, '，': 0.0, '影史': 0.0, '最賣': 0.0, '座': 0.0, '動畫': 0.0, '電影': 0.0, '系列': 0.0, '之一': 0.0, '和': 0.0, '風靡': 0.0, '全球': 0.0, '現象': 0.0, '即將': 0.0, '帶來關': 0.0, '於': 0.0, '一個': 0.0, '12': 0.0, '歲': 0.0, '小男孩': 0.0, '故事': 0.0, '他夢想': 0.0, '成為': 0.0, '世上': 0.0, '最偉大': 0.0, '超級': 0.0, '大壞': 0.0, '蛋': 0.0, '。': 0.0, '這部': 0.0, '全新': 0.0, '喜劇動': 0.0, '作片': 0.0, '就是': 0.0, '《': 0.0, '》': 0.0, '\r\n': 0.0, '\xa0': 0.0, '充滿': 0.0, '了': 0.0, '照明': 0.0, '娛樂': 0.0, '招牌': 0.0, '顛覆性': 0.0, '無厘頭': 0.0, '喜劇橋': 0.0, '段': 0.0, '、': 0.0, '各種': 0.0, '大眾': 0.0, '文化': 0.0, '笑': 0.0, '梗': 0.0, '感人肺腑': 0.0, '劇情': 0.0, '勁': 0.0, '爆': 0.0, '配樂': 0.0, '以及': 0.0, '宇宙': 0.0, '無敵': 0.0, '誇張': 0.0, '動作': 0.0, '場面': 0.0, '一片': 0.0, '由': 0.0, '這個': 0.0, '原創': 0.0, '掌舵': 0.0, '製': 0.0, '片': 0.0, '包括': 0.0, '娛樂極': 0.0, '具遠': 0.0, '見': 0.0, '創辦': 0.0, '人': 0.0, '執行長': 0.0, '克里斯': 0.0, '梅勒丹': 0.0, '德利': 0.0, '他': 0.0, '長': 0.0, '期': 0